<a href="https://colab.research.google.com/github/changcw2022/line_chatbot/blob/main/line_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 掛載google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 安裝套件
! pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.7 MB/s 


In [3]:
# 引用套件
from flask import Flask, request, abort, jsonify
import json
from flask_ngrok import run_with_ngrok
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)

In [4]:
# 建置app主程序
app = Flask(__name__, static_url_path = "/material", static_folder = "./material/")
run_with_ngrok(app)

line_bot_api = LineBotApi("QvLMuQywGLOlgbbWNaQNzjeb6+UJvn3T8TBFhfEb+AvWsU+SH0h3G3xk8xwJinz0rVakHW/gL8kQ4nfRsDwv2Qocuzp/Oz8+OpYtV+nFZYk1gbug5/Py+ZCXtHw3CHk9XD9SoTPEKYfd0ndxx3my7wdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("1caa479b27857d0b6e7d6cd93c5322ba")

In [5]:
# 建置主程序的api接口
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print(body)
    
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [6]:
# 告知handler，收到FollowEvent，則作以下的處理方式
# 1.儲存個資
# 2.回覆訊息

from linebot.models.events import FollowEvent
from linebot.models import (TextSendMessage, ImageSendMessage)


@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)

    with open("/content/drive/MyDrive/user.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile), sort_keys=True))
        myfile.write('\n')

    reply_array=[]
    reply_array.append(TextSendMessage('歡迎來到 好家在 yoga\U0001F3E0\n想學瑜珈，但不知從何開始，那你就來對地方啦\U0001F60A\n\n我們提供入門瑜珈課程介紹，讓你在家也能輕鬆做瑜珈'))
    reply_array.append(ImageSendMessage(original_content_url='https://pic.pimg.tw/yogiiilovestea/1636105025-3268922374-g_n.jpg',
                                        preview_image_url='https://pic.pimg.tw/yogiiilovestea/1636105025-3268922374-g_n.jpg'))
    reply_array.append(TextSendMessage('\U0001F449請回覆：我要學瑜珈\U0001F448\n\n讓我們一起開啟yoga之旅\nNamaste!\U0001F9D8 \U0001F917'))

    
    line_bot_api.reply_message(
        event.reply_token, reply_array
    )

In [7]:
# 創建quick reply buttons(快速回覆，按鈕按完就會消失)
# 以QuickReply封裝該些按鈕
# 製作TextSendMessage，放入剛封裝的QuickReply

from linebot.models import TextSendMessage
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton)


quick_reply_text_send_message = TextSendMessage(text='請告訴我你想做哪一種瑜珈？',
                                                quick_reply=QuickReply(
    items=[
        QuickReplyButton(
            action=PostbackAction(
            label='早晨yoga',
            display_text='我想做早晨yoga，開啟美好的一天',
            data='早起')),
        QuickReplyButton(
            action=PostbackAction(
            label='擊退腰痠背痛',
            display_text='我要擊退腰痠背痛，救救我的腰',
            data='上班族')),
        QuickReplyButton(
            action=PostbackAction(
            label='瘦小腹',
            display_text='我想瘦小腹，躺著練腹肌',
            data='凸肚')),
        QuickReplyButton(
            action=PostbackAction(
            label='睡前yoga',
            display_text='我想做睡前yoga，一覺到天亮',
            data='晚睡')),
        QuickReplyButton(
            action=PostbackAction(
            label='釋放負能量',
            display_text='我需要釋放負能量，來個舒壓儀式',
            data='壓力大')),
        QuickReplyButton(
            action=PostbackAction(
            label='親子yoga',
            display_text='我的小孩需要放電！！！',
            data='親子'))
    ]
                                                )
)

In [8]:
# 創建Carousel Template(每個CarouselColumn限制3個actions)
# 製作TemplateSendMessage，放入剛創建的template

from linebot.models import (MessageEvent, TemplateSendMessage , PostbackEvent, MessageAction)
from linebot.models.template import(ButtonsTemplate, CarouselTemplate, CarouselColumn)

buttons_template_message = TemplateSendMessage(
    alt_text='Carousel template',  
    template=CarouselTemplate(
        columns=[
            CarouselColumn(
                thumbnail_image_url='https://memegenerator.net/img/instances/58975815.jpg',
                title='初學者',
                text='請點擊下方選項選擇不同的瑜珈動作或介紹',
                actions=[
                   {
                        "type": "postback",
                        "label": "下犬式",
                        "text": "我想了解瑜珈動作：下犬式",
                        "data": "beginner"
                    },
                    {
                        "type": "postback",
                        "label": "嬰兒式",
                        "text": "我想了解瑜珈動作：嬰兒式",
                        "data": "beginner"
                    },
                    {
                        "type": "uri",
                        "label": "更多介紹",
                        "uri":"https://yas.io/hk/blog/lifestyle/%E7%91%9C%E7%8F%88%E5%88%9D%E5%AD%B8%E8%80%85%E6%8C%87%E5%8D%97-%E7%91%9C%E7%8F%88%E5%88%9D%E5%AD%B8%E5%8B%95%E4%BD%9C%E6%8E%A8%E8%96%A6/"
                    }
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://i.chzbgr.com/full/9596945664/h9D1630D4/cat-yoga-advanced-course',
                title='進階者',
                text='請點擊下方選項選擇不同的瑜珈動作或介紹',
                actions=[
                    {
                        "type": "postback",
                        "label": "輪式",
                        "text": "我想了解瑜珈動作：輪式",
                        "data": "advancer"
                    },
                    {
                        "type": "postback",
                        "label": "烏鴉式",
                        "text": "我想了解瑜珈動作：烏鴉式",
                        "data": "advancer"
                    },
                    {
                        "type": "uri",
                        "label": "瑜珈派別介紹",
                        "uri":"https://www.havfit.com/fitness/2qni0jhe"
                    }
                ]
            )
            ]
        )
)

In [9]:
practice_text_send_message = TextSendMessage(text="以下將隨機出現瑜珈動作，每日練習10分鐘，幫助伸展與放鬆身心\n\n你準備好了嗎？\n\n\U0001F449準備好的話，請輸入：我準備好了")

In [10]:
import random
original_content_url_list=[]
preview_image_url_list=[]

original_content={
    0:'https://pic.pimg.tw/yogiiilovestea/1636104962-1701745454-g_n.jpg',
    1:'https://pic.pimg.tw/yogiiilovestea/1636104962-1321811858-g_n.jpg',
    2:'https://img-9gag-fun.9cache.com/photo/aW6y0Zq_700bwp.webp'
    }

preview_image={
    0:'https://pic.pimg.tw/yogiiilovestea/1636104962-1701745454-g_n.jpg',
    1:'https://pic.pimg.tw/yogiiilovestea/1636104962-1321811858-g_n.jpg',
    2:'https://img-9gag-fun.9cache.com/photo/aW6y0Zq_700bwp.webp'
    }

i = random.randint(0,3)
original_content_url_list.append(original_content[2])
preview_image_url_list.append(original_content[2])


practice_img_send_message = ImageSendMessage(
    original_content_url=original_content_url_list[0],
    preview_image_url=preview_image_url_list[0]
)

In [11]:
# 設計字典，當用戶輸入相對應的消息時，會觸發相對的反應

from linebot.models import (MessageEvent, TextMessage, TextSendMessage, ImageSendMessage, VideoSendMessage)

template_message_dict = {
    '我要學瑜珈':quick_reply_text_send_message,
    '動作':buttons_template_message,
    '挑戰':practice_text_send_message,
    "我準備好了":practice_img_send_message
}

In [12]:
# 當handler收到PostbackEvent，會執行以下的處理方式
from linebot.models import PostbackEvent

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile=line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('早起')==0):
        with open('user_profile_morning_bird.txt', 'a') as myfile:
            myfile.write(json.dumps(vars(user_profile), sort_keys=True))
            myfile.write('\n')
            reply_array=[]
            reply_array.append(TextSendMessage('https://www.youtube.com/watch?v=E-H_WuIAR7I'))
            reply_array.append(TextSendMessage('希望你喜歡這部影片\U0001F60A\n\n\U0001F449若要學其他種瑜珈，請重新輸入：我要學瑜珈\n\n\U0001F449瑜珈動作介紹，請輸入：動作\n\n\U0001F449參加每日瑜珈小挑戰，請輸入：挑戰'))
            line_bot_api.reply_message(
                event.reply_token,reply_array
            )
    elif (event.postback.data.find('上班族')==0):
        with open('user_profile_officer.txt', 'a') as myfile:
            myfile.write(json.dumps(vars(user_profile), sort_keys=True))
            myfile.write('\n')
            reply_array=[]
            reply_array.append(TextSendMessage('https://www.youtube.com/watch?v=B6azLlYhUuI'))
            reply_array.append(TextSendMessage('希望你喜歡這部影片\U0001F60A\n\n\U0001F449若要學其他種瑜珈，請重新輸入：我要學瑜珈\n\n\U0001F449瑜珈動作介紹，請輸入：動作\n\n\U0001F449參加每日瑜珈小挑戰，請輸入：挑戰'))
            line_bot_api.reply_message(
                event.reply_token,reply_array
            )
    elif (event.postback.data.find('凸肚')==0):
        with open('user_profile_fat.txt', 'a') as myfile:
            myfile.write(json.dumps(vars(user_profile), sort_keys=True))
            myfile.write('\n')
            reply_array=[]
            reply_array.append(TextSendMessage('https://www.youtube.com/watch?v=_4vjR9pCGqg'))
            reply_array.append(TextSendMessage('希望你喜歡這部影片\U0001F60A\n\n\U0001F449若要學其他種瑜珈，請重新輸入：我要學瑜珈\n\n\U0001F449瑜珈動作介紹，請輸入：動作\n\n\U0001F449參加每日瑜珈小挑戰，請輸入：挑戰'))
            line_bot_api.reply_message(
                event.reply_token,reply_array
            )
    elif (event.postback.data.find('晚睡')==0):
        with open('user_profile_night_owal.txt', 'a') as myfile:
            myfile.write(json.dumps(vars(user_profile), sort_keys=True))
            myfile.write('\n')
            reply_array=[]
            reply_array.append(TextSendMessage('https://www.youtube.com/watch?v=s5Za2RHBUuk'))
            reply_array.append(TextSendMessage('希望你喜歡這部影片\U0001F60A\n\n\U0001F449若要學其他種瑜珈，請重新輸入：我要學瑜珈\n\n\U0001F449瑜珈動作介紹，請輸入：動作\n\n\U0001F449參加每日瑜珈小挑戰，請輸入：挑戰'))
            line_bot_api.reply_message(
                event.reply_token,reply_array
            )
    elif (event.postback.data.find('壓力大')==0):
        with open('user_profile_pressure', 'a') as myfile:
            myfile.write(json.dumps(vars(user_profile), sort_keys=True))
            myfile.write('\n')
            reply_array=[]
            reply_array.append(TextSendMessage('https://www.youtube.com/watch?v=VnWhDrk0ggI'))
            reply_array.append(TextSendMessage('希望你喜歡這部影片\U0001F60A\n\n\U0001F449若要學其他種瑜珈，請重新輸入：我要學瑜珈\n\n\U0001F449瑜珈動作介紹，請輸入：動作\n\n\U0001F449參加每日瑜珈小挑戰，請輸入：挑戰'))
            line_bot_api.reply_message(
                event.reply_token,reply_array
            )
    elif (event.postback.data.find('親子')==0):
        with open('user_profile_kids.txt', 'a') as myfile:
            myfile.write(json.dumps(vars(user_profile), sort_keys=True))
            myfile.write('\n')
            reply_array=[]
            reply_array.append(TextSendMessage('https://www.youtube.com/watch?v=_17BkaO62BU'))
            reply_array.append(TextSendMessage('希望你喜歡這部影片\U0001F60A\n\n\U0001F449若要學其他種瑜珈，請重新輸入：我要學瑜珈\n\n\U0001F449瑜珈動作介紹，請輸入：動作\n\n\U0001F449參加每日瑜珈小挑戰，請輸入：挑戰'))
            line_bot_api.reply_message(
                event.reply_token,reply_array
            )
    else:
        pass

In [13]:
# 當handler收到MessageEvent，會執行以下的處理方式

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if(event.message.text.find('我要學瑜珈')==0):
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(event.message.text)
        )
    elif (event.message.text.find('動作')==0):
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(event.message.text)
        )
    elif (event.message.text.find('我想了解瑜珈動作：嬰兒式')==0):
        line_bot_api.reply_message(
            event.reply_token,
            ImageSendMessage(original_content_url='https://www.verywellfit.com/thmb/Hz9tPl8mmSk3c3s-UbBgeO6cRBg=/500x350/filters:no_upscale():max_bytes(150000):strip_icc()/ChildsPose_annotated-3bdcd72451e84925af5a9fe8118935ed.jpg',
                            preview_image_url='https://www.verywellfit.com/thmb/Hz9tPl8mmSk3c3s-UbBgeO6cRBg=/500x350/filters:no_upscale():max_bytes(150000):strip_icc()/ChildsPose_annotated-3bdcd72451e84925af5a9fe8118935ed.jpg')
        )
    elif (event.message.text.find('我想了解瑜珈動作：下犬式')==0):
        line_bot_api.reply_message(
            event.reply_token,
            ImageSendMessage(original_content_url='https://www.verywellfit.com/thmb/i__eYpZwxa1meCPELijVi9a7460=/614x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/DownwardFacingDogAdhoMukahSvanasana_annotated2-918a8568b3af49d2add6d1949797a20f.jpg',
                            preview_image_url='https://www.verywellfit.com/thmb/i__eYpZwxa1meCPELijVi9a7460=/614x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/DownwardFacingDogAdhoMukahSvanasana_annotated2-918a8568b3af49d2add6d1949797a20f.jpg')
            
        )     
    elif (event.message.text.find('我想了解瑜珈動作：輪式')==0):
        line_bot_api.reply_message(
            event.reply_token,
            ImageSendMessage(original_content_url='https://www.verywellfit.com/thmb/MLJRHg0oA6hWaSjU7KxFBX6P6qQ=/800x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/55-3567138-WheelPose-004-b768aa1db0ff4ba99e01947e2ffb5b84.jpg',
                            preview_image_url='https://www.verywellfit.com/thmb/MLJRHg0oA6hWaSjU7KxFBX6P6qQ=/800x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/55-3567138-WheelPose-004-b768aa1db0ff4ba99e01947e2ffb5b84.jpg')   
        )
    elif (event.message.text.find('我想了解瑜珈動作：烏鴉式')==0):
        line_bot_api.reply_message(
            event.reply_token,
            ImageSendMessage(original_content_url='https://www.yogajournal.com/wp-content/uploads/2015/08/erin_motz_crow_pose.jpg?crop=535:301&width=1070&enable=upscale',
                            preview_image_url='https://www.yogajournal.com/wp-content/uploads/2015/08/erin_motz_crow_pose.jpg?crop=535:301&width=1070&enable=upscale')       
        )
    elif (event.message.text.find('挑戰')==0):
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(event.message.text)    
        )
    elif (event.message.text.find("我準備好了")==0):
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(event.message.text)    
        )
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='瑜珈教學，請輸入：我要學瑜珈\n\n瑜珈動作介紹，請輸入：動作\n\n參加每日瑜珈小挑戰，請輸入：挑戰')
        )

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bdb8-34-74-251-210.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U284954882fe05853b905769805e4bef5","events":[{"type":"unfollow","webhookEventId":"01GBM4GZNF5WET5CEP5W0BVPHK","deliveryContext":{"isRedelivery":false},"timestamp":1661754441198,"source":{"type":"user","userId":"U0caf4a65703ad0276464f072b0f13c47"},"mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Aug/2022 06:27:22] "POST / HTTP/1.1" 200 -


{"destination":"U284954882fe05853b905769805e4bef5","events":[{"type":"follow","webhookEventId":"01GBM4H1GQGTDGKZJAPT9REC80","deliveryContext":{"isRedelivery":false},"timestamp":1661754443065,"source":{"type":"user","userId":"U0caf4a65703ad0276464f072b0f13c47"},"replyToken":"c5b1d1473de6465899e355eb468c60ee","mode":"active"}]}


...

INFO:werkzeug:127.0.0.1 - - [29/Aug/2022 06:27:35] "POST / HTTP/1.1" 200 -


{"destination":"U284954882fe05853b905769805e4bef5","events":[{"type":"postback","postback":{"data":"凸肚"},"webhookEventId":"01GBM4HGDHMQ3JT0S0MMBFYJRS","deliveryContext":{"isRedelivery":false},"timestamp":1661754458238,"source":{"type":"user","userId":"U0caf4a65703ad0276464f072b0f13c47"},"replyToken":"2a645cb0fb1a43beba85142f6e39e32f","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Aug/2022 06:27:39] "POST / HTTP/1.1" 200 -


{"destination":"U284954882fe05853b905769805e4bef5","events":[{"type":"message","message":{"type":"text","id":"16689623747561","text":"挑戰"},"webhookEventId":"01GBM4J1Y76FYGG1XVW2541W92","deliveryContext":{"isRedelivery":false},"timestamp":1661754476347,"source":{"type":"user","userId":"U0caf4a65703ad0276464f072b0f13c47"},"replyToken":"188cee94b5c041b49d3442481de5bc0e","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Aug/2022 06:27:57] "POST / HTTP/1.1" 200 -
